In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Drop unnecesarry features

In [4]:
train1 = train.drop(['Ticket', 'Cabin'], axis='columns')
test1 = test.drop(['Ticket', 'Cabin'], axis='columns')

In [5]:
train1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


# Fill missing values

In [6]:
train1.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [7]:
train1.shape

(891, 10)

In [8]:
mode = train1.Embarked.dropna().mode()[0]
mode

'S'

In [9]:
train1.Embarked.fillna(mode, inplace=True)

In [10]:
test1.isnull().sum()

PassengerId     0
Pclass          0
Name            0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64

In [11]:
median = test1.Fare.dropna().median()
median

14.4542

In [12]:
test1.Fare.fillna(median, inplace=True)

In [13]:
combine = pd.concat([train1, test1], axis=0).reset_index(drop=True)
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [14]:
combine.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Fare             0
Embarked         0
dtype: int64

In [15]:
combine['Sex'] = combine['Sex'].map({'male':0, 'female':1})

In [16]:
age_nan_indices = list(combine[combine['Age'].isnull()].index)
len(age_nan_indices)

263

In [17]:
for index in age_nan_indices:
    median_age = combine['Age'].median()
    predict_age = combine['Age'][(combine['SibSp'] == combine.iloc[index]['SibSp'])
                                & (combine['Parch'] == combine.iloc[index]['Parch'])
                                & (combine['Pclass'] == combine.iloc[index]['Pclass'])].median()
    
    if np.isnan(predict_age):
        combine['Age'].iloc[index] = median_age
    else:
        combine['Age'].iloc[index] = predict_age

C:\Users\WINDOWS 10\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
combine.Age.isnull().sum()

0

# Data Transformation

In [19]:
combine['Fare'] = combine['Fare'].map(lambda x: np.log(x) if x>0 else 0) 

# Feature Engineering

In [20]:
combine.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,1.981001,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,4.266662,C
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,2.070022,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,3.972177,S
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,2.085672,S


In [21]:
combine['Title'] = [name.split(',')[1].split('.')[0].strip() for name in combine['Name']]
combine[['Name', 'Title']].head()

,Name,Title
0,"Braund, Mr. Owen Harris",Mr
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs
2,"Heikkinen, Miss. Laina",Miss
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Mrs
4,"Allen, Mr. William Henry",Mr


In [22]:
combine['Title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Mme               1
Don               1
Jonkheer          1
Sir               1
Lady              1
Dona              1
the Countess      1
Capt              1
Name: Title, dtype: int64

In [23]:
combine.Title.nunique()

18

In [24]:
combine[['Title', 'Survived']].groupby(['Title'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Title,Survived
17,the Countess,1.000000
10,Mlle,1.000000
16,Sir,1.000000
14,Ms,1.000000
6,Lady,1.000000
11,Mme,1.000000
13,Mrs,0.792000
9,Miss,0.697802
8,Master,0.575000
1,Col,0.500000


In [25]:
combine = combine.drop('Name', axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,0,22.0,1,0,1.981001,S,Mr
1,2,1.0,1,1,38.0,1,0,4.266662,C,Mrs
2,3,1.0,3,1,26.0,0,0,2.070022,S,Miss
3,4,1.0,1,1,35.0,1,0,3.972177,S,Mrs
4,5,0.0,3,0,35.0,0,0,2.085672,S,Mr


## is alone

In [26]:
combine['FamilySize'] = combine['SibSp'] + combine['Parch'] + 1
combine[['SibSp', 'Parch', 'FamilySize']].head(10)

,SibSp,Parch,FamilySize
0,1,0,2
1,1,0,2
2,0,0,1
3,1,0,2
4,0,0,1
5,0,0,1
6,0,0,1
7,3,1,5
8,0,2,3
9,1,0,2


In [27]:
combine[['FamilySize', 'Survived']].groupby('FamilySize', as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [28]:
combine['IsAlone'] = 0
combine.loc[combine['FamilySize'] == 1, 'IsAlone'] = 1

In [29]:
combine[['IsAlone', 'Survived']].groupby('IsAlone', as_index=False).mean().sort_values(by='Survived', ascending=False)

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [30]:
combine = combine.drop(['SibSp', 'Parch', 'FamilySize'], axis=1)
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,1,0.0,3,0,22.0,1.981001,S,Mr,0
1,2,1.0,1,1,38.0,4.266662,C,Mrs,0
2,3,1.0,3,1,26.0,2.070022,S,Miss,1
3,4,1.0,1,1,35.0,3.972177,S,Mrs,0
4,5,0.0,3,0,35.0,2.085672,S,Mr,1


## Age and Class

In [31]:
combine['AgeBand'] = pd.cut(combine['Age'], 5)
combine[['AgeBand', 'Survived']].groupby('AgeBand', as_index=False).mean().sort_values(by='AgeBand')

,AgeBand,Survived
0,"(0.0902, 16.136]",0.500000
1,"(16.136, 32.102]",0.341719
2,"(32.102, 48.068]",0.412844
3,"(48.068, 64.034]",0.434783
4,"(64.034, 80.0]",0.090909


In [32]:
combine.loc[combine['Age'] <= 16.136, 'Age'] = 0
combine.loc[(combine['Age'] > 16.136) & (combine['Age'] <= 32.102), 'Age'] = 1
combine.loc[(combine['Age'] > 32.102) & (combine['Age'] <= 48.068), 'Age'] = 2
combine.loc[(combine['Age'] > 48.068) & (combine['Age'] <= 64.034), 'Age'] = 3
combine.loc[combine['Age'] > 64.034, 'Age'] =4

In [33]:
combine = combine.drop('AgeBand', axis=1)

In [34]:
combine[['Age', 'Pclass']].dtypes

Age       float64
Pclass      int64
dtype: object

In [35]:
combine['Age'] = combine['Age'].astype('int')
combine['Age'].dtype

dtype('int32')

In [36]:
combine['Age*Class'] = combine['Age'] * combine['Pclass']
combine[['Age', 'Pclass', 'Age*Class']].head()

,Age,Pclass,Age*Class
0,1,3,3
1,2,1,2
2,1,3,3
3,2,1,2
4,2,3,6


# Feature Encoding

In [37]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,1,0.0,3,0,1,1.981001,S,Mr,0,3
1,2,1.0,1,1,2,4.266662,C,Mrs,0,2
2,3,1.0,3,1,1,2.070022,S,Miss,1,3
3,4,1.0,1,1,2,3.972177,S,Mrs,0,2
4,5,0.0,3,0,2,2.085672,S,Mr,1,6


In [38]:
combine = pd.get_dummies(combine, columns=['Title'])
combine = pd.get_dummies(combine, columns=['Embarked'], prefix='Em')
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1.981001,0,3,0,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1.0,1,1,2,4.266662,0,2,0,0,...,0,0,1,0,0,0,0,1,0,0
2,3,1.0,3,1,1,2.070022,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,1,1,2,3.972177,0,2,0,0,...,0,0,1,0,0,0,0,0,0,1
4,5,0.0,3,0,2,2.085672,1,6,0,0,...,0,1,0,0,0,0,0,0,0,1


In [39]:
combine['FareBand'] = pd.cut(combine['Fare'], 4)
combine[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand')

,FareBand,Survived
0,"(-0.00624, 1.56]",0.062500
1,"(1.56, 3.119]",0.288719
2,"(3.119, 4.679]",0.504983
3,"(4.679, 6.239]",0.745098


In [40]:
combine.loc[combine['Fare'] <= 1.56, 'Fare'] = 0
combine.loc[(combine['Fare'] > 1.56) & (combine['Fare'] <= 3.119), 'Fare'] = 1
combine.loc[(combine['Fare'] > 3.119) & (combine['Fare'] <= 4.679), 'Fare'] = 2
combine.loc[combine['Fare'] > 4.679, 'Fare'] = 3

In [41]:
combine['Fare'] = combine['Fare'].astype('int')

In [42]:
combine = combine.drop('FareBand', axis=1)

In [43]:
combine.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1,0,3,0,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1.0,1,1,2,2,0,2,0,0,...,0,0,1,0,0,0,0,1,0,0
2,3,1.0,3,1,1,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,1,1,2,2,0,2,0,0,...,0,0,1,0,0,0,0,0,0,1
4,5,0.0,3,0,2,1,1,6,0,0,...,0,1,0,0,0,0,0,0,0,1


In [44]:
train_set = combine[:len(train1)]
test_set = combine[len(train1):]

In [45]:
train_set.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
0,1,0.0,3,0,1,1,0,3,0,0,...,0,1,0,0,0,0,0,0,0,1
1,2,1.0,1,1,2,2,0,2,0,0,...,0,0,1,0,0,0,0,1,0,0
2,3,1.0,3,1,1,1,1,3,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,1,1,2,2,0,2,0,0,...,0,0,1,0,0,0,0,0,0,1
4,5,0.0,3,0,2,1,1,6,0,0,...,0,1,0,0,0,0,0,0,0,1


In [46]:
train_set.shape

(891, 29)

In [47]:
train_set = train_set.drop('PassengerId', axis=1)
train_set.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,Title_Don,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
0,0.0,3,0,1,1,0,3,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1.0,1,1,2,2,0,2,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,1.0,3,1,1,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.0,1,1,2,2,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0.0,3,0,2,1,1,6,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [48]:
train_set['Survived'] = train_set['Survived'].astype('int')

In [49]:
train_set.head()

,Survived,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,Title_Don,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
0,0,3,0,1,1,0,3,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1,1,1,2,2,0,2,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,1,3,1,1,1,1,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,2,2,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,0,3,0,2,1,1,6,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [50]:
test_set = test_set.drop('Survived', axis=1)
test_set.head()

,PassengerId,Pclass,Sex,Age,Fare,IsAlone,Age*Class,Title_Capt,Title_Col,Title_Don,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Em_C,Em_Q,Em_S
891,892,3,0,2,1,1,6,0,0,0,...,0,1,0,0,0,0,0,0,1,0
892,893,3,1,2,1,0,6,0,0,0,...,0,0,1,0,0,0,0,0,0,1
893,894,2,0,3,1,1,6,0,0,0,...,0,1,0,0,0,0,0,0,1,0
894,895,3,0,1,1,1,3,0,0,0,...,0,1,0,0,0,0,0,0,0,1
895,896,3,1,1,1,0,3,0,0,0,...,0,0,1,0,0,0,0,0,0,1


# Modelling

## 1.Splitting training data

In [51]:
X_train = train_set.drop('Survived', axis=1)
y_train = train_set['Survived']
X_test = test_set.drop('PassengerId', axis=1).copy()
print('X_train shape: ', X_train.shape)
print('y_train shape', y_train.shape)
print('X_test shape', X_test.shape)

X_train shape:  (891, 27)
y_train shape (891,)
X_test shape (418, 27)


## 2.Fit the Model

### LogisticRegression

In [52]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

acc_log = round(log_reg.score(X_train, y_train) * 100, 2)
acc_log

82.94

### DecisionTreeRegressor

In [53]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)
y_pred = tree_clf.predict(X_test)

acc_tree_clf = round(tree_clf.score(X_train, y_train) * 100, 2)
acc_tree_clf

85.97

### RandomForestClassifier

In [54]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

85.97

## 3.Model Evaluation and Hyperparameter Tuning

### Training accuracy

In [55]:
models = pd.DataFrame({'Model': ['LogisticRegression', 'DecisionTreeRegressor', 'RandomForestClassifier'],
                       'Score': [acc_log, acc_tree_clf, acc_random_forest]})
models.sort_values(by='Score', ascending=False, ignore_index=True)

,Model,Score
0,DecisionTreeRegressor,85.97
1,RandomForestClassifier,85.97
2,LogisticRegression,82.94


### K-fold Cross Validation

In [56]:
classifiers = []
classifiers.append(LogisticRegression())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
len(classifiers)

3

In [57]:
from sklearn.model_selection import cross_val_score

In [58]:
cv_results = []
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier, X_train, y_train, scoring = 'accuracy', cv=3))
    print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurit

In [59]:
cv_mean = []
cv_std = []
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())

In [60]:
cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 
                       'Algorithm': ['LogisticRegression', 'DecisionTreeRegressor', 'RandomForestClassifier']})
cv_res.sort_values(by='Cross Validation Mean', ascending=False, ignore_index=True)

,Cross Validation Mean,Algorithm
0,0.806958,RandomForestClassifier
1,0.805836,DecisionTreeRegressor
2,0.802469,LogisticRegression


In [61]:
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [62]:
len(y_pred)

418

# Preparing the data for submission

In [63]:
submit = pd.DataFrame({'PassengerId': test_set['PassengerId'], 'Survived': y_pred})
submit.head()

,PassengerId,Survived
891,892,0
892,893,1
893,894,0
894,895,0
895,896,1


In [64]:
submit.shape

(418, 2)